In [1]:
import pandas as pd
import numpy as np
import duckdb
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

# Suprimir warnings
warnings.filterwarnings('ignore')

# Configurações Visuais
sns.set_style('whitegrid')

# Caminhos
# Ajuste o nome do arquivo se for diferente (ex: IBGE.xls ou IBGE.xlsx)
RAW_FILE = Path('../data/raw/atlas/IBGE.xlsx') 
PROCESSED_PATH = Path('../data/processed')
PROCESSED_PATH.mkdir(exist_ok=True, parents=True)

print("Ambiente configurado.")

Ambiente configurado.


Como o arquivo bruto IBGE.xlsx está no formato wide, isso significa que os anos estão nas colunas, tipo: "População de 2010", eu vou transformar ele para deixar no padrão. Essa transformação leva o nome de **"MELT"**

In [ ]:
def carregar_demografia_wide(arquivo):
    print(f'Carregando: {arquivo}')
    
    try:
        # Tenta ler Excel ou CSV (Atlas as vezes salva como csv mas com nome xlsx)
        try:
            df = pd.read_excel(arquivo)
        except:
            df = pd.read_csv(arquivo, sep=';', encoding='latin1', decimal=',')
    except Exception as e:
        print(f'[ERRO] Falha ao ler arquivo: {e}')
        return pd.DataFrame()
    
    # A primeira coluna é sempre o local (Territorialidades)
    col_local = df.columns[0] 
    print(f"Coluna de local identificada: {col_local}")

    dfs_anos = []

    mapa_anos = {
        2000: {
            'População total 2000': 'populacao',
            'População Urbana 2000': 'populacao_urbana', 
            'Taxa de urbanização 2000': 'tx_urbanizacao',
            'Taxa de envelhecimento 2000': 'tx_envelhecimento'
        },
        2010: {
            'População total 2010': 'populacao',
            'População Urbana 2010': 'populacao_urbana',
            'Taxa de urbanização 2010': 'tx_urbanizacao',
            'Taxa de envelhecimento 2010': 'tx_envelhecimento'
        },
        2015: { # Se existir
            'População total 2015': 'populacao',
            'População Urbana 2015': 'populacao_urbana',
            'Taxa de urbanização 2015': 'tx_urbanizacao',
            'Taxa de envelhecimento 2015': 'tx_envelhecimento'
        }
    }

    for ano, colunas_alvo in mapa_anos.items():
        # Verifica quais dessas colunas realmente existem no arquivo
        cols_existentes = [c for c in colunas_alvo.keys() if c in df.columns]
        
        if cols_existentes:
            # Seleciona só o Nome da Cidade + Colunas desse Ano
            subset = df[[col_local] + cols_existentes].copy()
            
            # Renomeia as colunas para o padrão limpo (sem o ano no nome)
            subset = subset.rename(columns=colunas_alvo)
            
            # Renomeia a coluna de local para padrao
            subset = subset.rename(columns={col_local: 'nome_origem'})
            
            # Cria a coluna ANO
            subset['ano'] = ano
            
            # Se tiver população urbana mas nao a taxa, calcula a taxa agora
            if 'populacao_urbana' in subset.columns and 'populacao' in subset.columns and 'tx_urbanizacao' not in subset.columns:
                 subset['tx_urbanizacao'] = (subset['populacao_urbana'] / subset['populacao']) * 100
            
            dfs_anos.append(subset)
            print(f"   Ano {ano} processado com {len(cols_existentes)} indicadores.")

    # Junta tudo num bloco só (formato Tidy/Long)
    if dfs_anos:
        df_tidy = pd.concat(dfs_anos, ignore_index=True)
        return df_tidy
    else:
        print("[ERRO] Nenhuma coluna esperada foi encontrada!")
        print("Colunas no arquivo:", df.columns.tolist())
        return pd.DataFrame()

# Teste imediato
df_demo_raw = carregar_demografia_wide(RAW_FILE)
print(f'Shape inicial: {df_demo_raw.shape}')
df_demo_raw.head()